A heuristic approach to detect opinion targets in sentences.



References:
- An Unsupervised Aspect Detection Model for Sentiment Analysis of Reviews (https://link.springer.com/chapter/10.1007/978-3-642-38824-8_12)
- A Review of Unsupervised Approaches of Opinion Target Extraction from
Unstructured Reviews (https://www.researchgate.net/profile/Khairullah-Khan/publication/288148333_A_Review_of_Unsupervised_Approaches_of_Opinion_Target_Extraction_from_Unstructured_Reviews/links/5808a2ba08aefaf02a2d514f/A-Review-of-Unsupervised-Approaches-of-Opinion-Target-Extraction-from-Unstructured-Reviews.pdf)
- Amazon review data set. (https://www.kaggle.com/datafiniti/consumer-reviews-of-amazon-products)

In [1]:
import pandas as pd
import nltk
import numpy as np
import spacy

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mokarakaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
df = pd.read_csv("~/develop/dataset/kaggle-consumer-reviews-of-amazon-products/1429_1.csv")

/Users/mokarakaya/opt/anaconda3/envs/develop/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
sentences = []
for idx, row in df[1:2000].iterrows():
    text = row["reviews.text"]
    s = nltk.sent_tokenize(text)
    sentences.extend(s)

In [6]:
# for s in sentences:
#     doc = nlp(s)
#     print(s)
#     for token in doc:
#         print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#             token.shape_, token.is_alpha, token.is_stop)
#     print()

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
#vectorizer = CountVectorizer()
vec = CountVectorizer(analyzer='word', ngram_range=(1, 4)).fit(sentences)
bag_of_words = vec.transform(sentences)
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

In [9]:
def has_noun(doc):
    for token in doc:
        if token.pos_ in ["PROPN", "NOUN"]:
            return True
    return Falseq
def number_of_nouns(doc):
    count = 0
    for token in doc:
        if token.pos_ in ["PROPN", "NOUN"]:
            count +=1
    return count
def has_verb(doc):
    for token in doc:
        if token.pos_ in ["VERB"]:
            return True
    return False
def ends_with_verb(doc):
    return doc[-1].pos_ == "VERB" and doc[-1].dep_== "ROOT"

def ends_with_noun(doc):
    return doc[-1].pos_ in ["PROPN", "NOUN"]


def has_adj(doc):
    for token in doc:
        if token.pos_ in ["ADJ"]:
            return True
    return False

def is_selected(doc):
    # if len(doc) == 1:
    #     return ends_with_noun(doc)
    # else:
    #     if ends_with_noun(doc):
    #         return True
    #     elif ends_with_verb(doc):
    #         return has_adj(doc) or has_noun(doc)
    if ends_with_noun(doc) or ends_with_verb(doc):
        if sum([has_adj(doc),has_noun(doc), has_verb(doc)]) > 1:
            return True
        return number_of_nouns(doc) > 1
    return False


for s, count in words_freq[:1000]:
    doc = nlp(s)
    selected = is_selected(doc)
    if not selected:
        continue
    print(s, count)
    print("!!tags!!!")
    print(f"selected:{selected}")
    for token in doc:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)


    print()

kindle fire 109
!!tags!!!
selected:True
kindle kindle PROPN NNP compound xxxx True False
fire fire PROPN NNP ROOT xxxx True False

great tablet 104
!!tags!!!
selected:True
great great ADJ JJ amod xxxx True False
tablet tablet NOUN NN ROOT xxxx True False

fire hd 70
!!tags!!!
selected:True
fire fire NOUN NN compound xxxx True False
hd hd PROPN NNP ROOT xx True False

amazon fire 66
!!tags!!!
selected:True
amazon amazon PROPN NNP compound xxxx True False
fire fire PROPN NNP ROOT xxxx True False

battery life 62
!!tags!!!
selected:True
battery battery NOUN NN compound xxxx True False
life life PROPN NNP ROOT xxxx True False

black friday 57
!!tags!!!
selected:True
black black PROPN NNP amod xxxx True False
friday friday PROPN NNP ROOT xxxx True False

great price 54
!!tags!!!
selected:True
great great ADJ JJ amod xxxx True False
price price NOUN NN ROOT xxxx True False

amazon prime 51
!!tags!!!
selected:True
amazon amazon PROPN NNP compound xxxx True False
prime prime PROPN NNP ROOT xxx

In [10]:
# doc = nlp("easy to use")
# for token in doc:
    # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        # token.shape_, token.is_alpha, token.is_stop)